#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [83]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

# 資料準備

In [84]:
#取得Keras Dataset
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [85]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [86]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [87]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [88]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [89]:
model = Sequential()

In [90]:
#卷積層1

In [91]:
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))

In [92]:
# model.add(Dropout(rate=0.25))

In [93]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [94]:
#卷積層2與池化層2

In [95]:
# model.add(Conv2D(filters=64, kernel_size=(3, 3), 
#                  activation='relu', padding='same'))

In [96]:
# model.add(Dropout(0.25))

In [97]:
# model.add(MaxPooling2D(pool_size=(2, 2)))

In [98]:
#建立神經網路(平坦層、隱藏層、輸出層)

In [99]:
model.add(Flatten())
model.add(Dropout(rate=0.25))

In [100]:
# model.add(Dense(1024, activation='relu'))
# # model.add(Dropout(rate=0.25))

In [101]:
model.add(Dense(10, activation='softmax'))

In [102]:
#檢查model 的STACK
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 8192)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                81930     
Total params: 82,826
Trainable params: 82,826
Non-trainable params: 0
_________________________________________________________________
None


# 載入之前訓練的模型

In [63]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

In [81]:
'''
作業:
請分別選用 "MSE", "binary _crossentropy"
查看Train/test accurancy and loss rate
'''
import pandas as pd

df = pd.DataFrame(columns=["loss_func", "acc", "val_acc", "loss", "val_loss"])
scores_list=list()

loss_func=["MSE", "categorical_crossentropy", "binary_crossentropy"]
epochs=2
for lf in loss_func:
    print("Perform loss funcion {}".format(lf))
    model.compile(loss=lf, optimizer='adam', metrics=['accuracy'])
    train_history = model.fit(x=x_img_train_normalize, y=y_label_train_OneHot, validation_split=0.25, epochs=epochs, batch_size=32, verbose=1) 
    for i in range(epochs):
        df = df.append({"loss_func":lf}, ignore_index=True)
    df = df.append({"acc": train_history.history["acc"]}, ignore_index=True)
    df = df.append({"val_acc": train_history.history["val_acc"]}, ignore_index=True)
    df = df.append({"loss": train_history.history["loss"]}, ignore_index=True)
    df = df.append({"val_loss": train_history.history["val_loss"]}, ignore_index=True)
    
    scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
    scores_list.append(scores[1])
    print("score={}".format(scores[1]))

Perform loss funcion MSE
Train on 37500 samples, validate on 12500 samples
Epoch 1/2
37500/37500 [==============================] - 699s 19ms/step - loss: 0.0046 - acc: 0.9704 - val_loss: 0.0470 - val_acc: 0.7146
Epoch 2/2
10000/10000 [==============================] - 57s 6ms/step
score=0.7014
Perform loss funcion categorical_crossentropy
Train on 37500 samples, validate on 12500 samples
Epoch 1/2
37500/37500 [==============================] - 593s 16ms/step - loss: 0.0886 - acc: 0.9742 - val_loss: 1.6348 - val_acc: 0.7133
Epoch 2/2
10000/10000 [==============================] - 63s 6ms/step
score=0.7126
Perform loss funcion binary_crossentropy
Train on 37500 samples, validate on 12500 samples
Epoch 1/2
37500/37500 [==============================] - 594s 16ms/step - loss: 0.0142 - acc: 0.9955 - val_loss: 0.3018 - val_acc: 0.9413
Epoch 2/2
10000/10000 [==============================] - 65s 6ms/step
score=0.9419300254821777


In [111]:
xxx = list()
xxx.append(train_history.history["val_loss"])
xxx.append(train_history.history["loss"])
xx = np.array(xxx)
xx = xx.reshape(1,-1)
xx.shape

(1, 4)

In [103]:
scores_list

[0.7014, 0.7126, 0.9419300254821777]